In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
## initialising constants for the objective function
pi = 22/7
h = 6.62e-34

In [3]:
## Generate integer population
n_population = 100
n_survivors = 5
integer_population = [random.randint(1,250) for i in range(n_population)]

In [4]:
## Generate float population
float_population = [np.random.normal(100,50) for i in range(n_population)]

In [5]:
## initialise the fitness/objective function
def check_fitness(m,E,V,psi):
    pi = 22/7
    h = 6.62e-34
    score = (8* pi ** 2 * m)/ (h ** 2) * (E-V) * psi
    return score

In [6]:
## generating first generation randomly
def generate_first_generation(population, n, encoding_length):
    first_gen = []
    for i in range(n):
        parents = []
        for j in range(encoding_length):
            index = random.randint(0,len(population) - 1)
            parents.append(population[index])
        first_gen.append(parents)
    return first_gen

In [7]:
## Crossover or Recombination
def crossover(parent_1, parent_2, typ='uniform', n = 1):
    if typ=='multi' and n > len(parent_1):
        print("Multi N crossover not possible as N > length of parent")
        return
    child_1 = []
    child_2 = []
    if typ!='uniform':
        split_points = []
        while len(split_points) != n:
            index = random.randint(1,len(parent_1) - 1)
            if index not in split_points:
                split_points.append(index)
        split_points.sort()
        prev_point = 0
        for i in range(n):
            if i%2 == 0:
                child_1.extend(parent_1[prev_point:split_points[i]])
                child_2.extend(parent_2[prev_point:split_points[i]])
            else:
                child_1.extend(parent_2[prev_point:split_points[i]])
                child_2.extend(parent_1[prev_point:split_points[i]])
            prev_point = split_points[i]
        if len(child_1) < len(parent_1):
            if n%2 == 0:
                child_1.extend(parent_1[prev_point:len(parent_1)])
                child_2.extend(parent_2[prev_point:len(parent_1)])
            else:
                child_1.extend(parent_2[prev_point:len(parent_1)])
                child_2.extend(parent_1[prev_point:len(parent_1)])
    else:
        for i in range(len(parent_1)):
            prob = np.random.normal(0.5,0.5)
            if prob > 0.5:
                child_1.append(parent_1[i])
                child_2.append(parent_2[i])
            else:
                child_1.append(parent_2[i])
                child_2.append(parent_1[i])
    return child_1, child_2

In [8]:
def mutate(child, population, n = 1, mode = 'uniform'):
    if mode == 'uniform':
        for i in range(len(child)):
            prob = np.random.normal(0.5,0.5)
            if prob > 0.5:
                pop_index = random.randint(0, len(population) - 1)
                child_index = random.randint(0, len(child) - 1)
                child[child_index] = population[pop_index]
    elif mode == 'non-uniform':
        for i in range(len(child)):
            prob = np.random.normal(0.5,0.5)
            if prob > 1:
                pop_index = random.randint(0, len(population) - 1)
                child_index = random.randint(0, len(child) - 1)
                child[child_index] = population[pop_index]
    return child

In [9]:
first_gen = generate_first_generation(float_population, 10, 4)
first_gen_df = pd.DataFrame(first_gen, columns = ['Mass','Total Energy','Potential Energy','Psi'])
first_gen_df

,Mass,Total Energy,Potential Energy,Psi
0,133.362090,-2.246444,107.901408,127.382819
1,66.292763,98.281729,69.175945,91.927839
2,115.358394,62.308844,144.575808,120.218703
3,166.512510,12.389385,78.812826,43.448668
4,129.838713,129.563730,176.575920,122.147711
5,166.512510,69.175945,166.512510,115.650550
6,15.461212,126.812951,157.958628,83.213881
7,78.812826,122.147711,129.563730,153.063001
8,55.263898,87.180954,217.822216,15.461212
9,-27.135806,92.871296,-27.135806,126.812951


In [10]:
## Evaluating the first generation
fitness = []
for i,r in first_gen_df.iterrows():
    fitness.append(check_fitness(r[0],r[1],r[2],r[3]))
first_gen_df['Fitness'] = fitness
first_gen_df

,Mass,Total Energy,Potential Energy,Psi,Fitness
0,133.362090,-2.246444,107.901408,127.382819,-3.373981e+74
1,66.292763,98.281729,69.175945,91.927839,3.198275e+73
2,115.358394,62.308844,144.575808,120.218703,-2.057169e+74
3,166.512510,12.389385,78.812826,43.448668,-8.664988e+73
4,129.838713,129.563730,176.575920,122.147711,-1.344384e+74
5,166.512510,69.175945,166.512510,115.650550,-3.379822e+74
6,15.461212,126.812951,157.958628,83.213881,-7.225375e+72
7,78.812826,122.147711,129.563730,153.063001,-1.613100e+73
8,55.263898,87.180954,217.822216,15.461212,-2.012745e+73
9,-27.135806,92.871296,-27.135806,126.812951,-7.446232e+73


In [11]:
first_gen_df = first_gen_df.sort_values(by = ['Fitness'], ascending = False)
first_gen_df.reset_index().drop(['index'], axis = 1)
first_gen_df

,Mass,Total Energy,Potential Energy,Psi,Fitness
1,66.292763,98.281729,69.175945,91.927839,3.198275e+73
6,15.461212,126.812951,157.958628,83.213881,-7.225375e+72
7,78.812826,122.147711,129.563730,153.063001,-1.613100e+73
8,55.263898,87.180954,217.822216,15.461212,-2.012745e+73
9,-27.135806,92.871296,-27.135806,126.812951,-7.446232e+73
3,166.512510,12.389385,78.812826,43.448668,-8.664988e+73
4,129.838713,129.563730,176.575920,122.147711,-1.344384e+74
2,115.358394,62.308844,144.575808,120.218703,-2.057169e+74
0,133.362090,-2.246444,107.901408,127.382819,-3.373981e+74
5,166.512510,69.175945,166.512510,115.650550,-3.379822e+74


In [12]:
parent_1 = first_gen_df.values[0][:-1]
parent_2 = first_gen_df.values[1][:-1]
child_1, child_2 = crossover(parent_1, parent_2, typ='uniform', n = 2)
print("Parent 1", parent_1)
print("Parent 2", parent_2)
print("\n")
print("After Crossover")
print("\n")
print("Child 1", child_1, end = "  =>  ")
print(check_fitness(child_1[0],child_1[1],child_1[2],child_1[3]))
print("Child 2", child_2, end = "  =>  ")
print(check_fitness(child_2[0],child_2[1],child_2[2],child_2[3]))
print("\n")
print("After Mutation")
mutated_child_1 = mutate(child_1,float_population)
mutated_child_2 = mutate(child_2,float_population)
print("\n")
print("Mutated Child 1", mutated_child_1, end = "  =>  ")
print(check_fitness(mutated_child_1[0],mutated_child_1[1],mutated_child_1[2],mutated_child_1[3]))
print("Mutated Child 2", mutated_child_2, end = "  =>  ")
print(check_fitness(mutated_child_2[0],mutated_child_2[1],mutated_child_2[2],mutated_child_2[3]))

Parent 1 [66.29276332 98.28172888 69.17594459 91.92783884]
Parent 2 [ 15.46121214 126.81295115 157.95862815  83.21388084]


After Crossover


Child 1 [66.29276331762509, 126.81295115308129, 157.9586281466153, 91.92783884430342]  =>  -3.4224273482704625e+73
Child 2 [15.461212137966797, 98.28172888451184, 69.17594458772487, 83.21388084040744]  =>  6.75214736018671e+72


After Mutation


Mutated Child 1 [64.46444547589218, 126.81295115308129, 157.9586281466153, 149.85675580945053]  =>  -5.425223605395281e+73
Mutated Child 2 [15.461212137966797, 98.28172888451184, 122.14771053502682, 184.8473800485638]  =>  -1.2298706303446458e+73
